In [1]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib


In [2]:
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

In [3]:
import pandas as pd
import time

In [4]:
df = pd.read_csv('./../data/nyc_bar_crawl_sample.csv')

In [5]:
# Yelp Fusion API

API_KEY= '4nRS21VgE79PQdVbjIz8wYPndzvh94hFffLad8WesROrkUBAqMftFyEUrcHlD1jLLBb6aggJD6NAEZcFOLjTn2hi3DGTM-jpiSisM8Aq4REo92Jwk6znbI-qKvAhY3Yx' 

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3


In [31]:

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

#     print(u'{0} businesses found, querying business info ' \
#         'for the top result "{1}" ...'.format(
#             len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
#     print(response)
#     pprint.pprint(response, indent=2)
    
#     pprint.pprint(response, indent=2)
    
    return response




In [46]:
def main():
    parser = argparse.ArgumentParser()

#     parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM,
#                         type=str, help='Search term (default: %(default)s)')
#     parser.add_argument('-l', '--location', dest='location',
#                         default=DEFAULT_LOCATION, type=str,
#                         help='Search location (default: %(default)s)')
#     parser.add_argument('-lo', '--longitude', dest='longitude',
#                         type=str, help='Search longitude')
#     parser.add_argument('-la', '--latitude', dest='latitude',
#                         type=str, help='Search latitude')

#     input_values = parser.parse_args()

    location = 'New York'

    for i in range(len(df)):
        print(df['bar_pub_name'][i])
        term = df['bar_pub_name'][i]
    
        try:
            response = query_api(term, location)
    #         query_api(input_values.term, input_values.location)
        except HTTPError as error:
            sys.exit(
                'Encountered HTTP error {0} on {1}:\n {2}\nAbort program.'.format(
                    error.code,
                    error.url,
                    error.read(),
                )
            )
            
        for res in response:
            if res == 'hours':
                process_hours(response['hours'])


In [57]:
def process_hours(hours):
    
    
    print("Type hours: ",type(hours))
    for h in hours:
        print("Type h: ",type(h))
        for io in h:
            print("Type io: ",type(io))
#             print("Key: ",io)
#             print("Value: ")
#             pprint.pprint(h[io])
            
            if io == 'open':
                for o in h[io]:
                    print(o)
            

In [16]:
if __name__ == '__main__':
    main()

310 BOWERY BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/RpgITHQX66KTVk3bN0w6dw ...
Result for business "RpgITHQX66KTVk3bN0w6dw" found:
{ 'alias': '310-bowery-bar-new-york',
  'categories': [ {'alias': 'venues', 'title': 'Venues & Event Spaces'},
                  {'alias': 'lounges', 'title': 'Lounges'}],
  'coordinates': {'latitude': 40.7248881372237, 'longitude': -73.9925056790985},
  'display_phone': '(917) 409-3036',
  'hours': [ { 'hours_type': 'REGULAR',
               'is_open_now': False,
               'open': [ { 'day': 0,
                           'end': '0200',
                           'is_overnight': True,
                           'start': '1500'},
                         { 'day': 1,
                           'end': '0200',
                           'is_overnight': True,
                           'start': '1500'},
                         { 'day': 2,
                           'end': '0200',
               

In [58]:
if __name__ == '__main__':
    main()

310 BOWERY BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/RpgITHQX66KTVk3bN0w6dw ...
Result for business "RpgITHQX66KTVk3bN0w6dw" found:
Type hours:  <class 'list'>
Type h:  <class 'dict'>
Type io:  <class 'str'>
{'is_overnight': True, 'start': '1500', 'end': '0200', 'day': 0}
{'is_overnight': True, 'start': '1500', 'end': '0200', 'day': 1}
{'is_overnight': True, 'start': '1500', 'end': '0200', 'day': 2}
{'is_overnight': True, 'start': '1500', 'end': '0400', 'day': 3}
{'is_overnight': True, 'start': '1500', 'end': '0400', 'day': 4}
{'is_overnight': True, 'start': '1200', 'end': '0400', 'day': 5}
{'is_overnight': True, 'start': '1200', 'end': '0200', 'day': 6}
Type io:  <class 'str'>
Type io:  <class 'str'>
55 BAR
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/9Ke2V1SJVOPyr4z3Bj4_IQ ...
Result for business "9Ke2V1SJVOPyr4z3Bj4_IQ" found:
Type hours:  <class 'list'>
Type h:  <class